In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\optimized_formula_all_variables_expand_WEIO_surface_deepchazhi_1.npz')
print(
    data.files)  # ['sst_obser', 'slfh', 'sshf', 'ssr', 'str', 'mld', 'analysis_temp', 'u', 'v', 'T_d', 'u_d', 'v_d', 'xx', 'yy', 'dT_dt', 'dT_dx', 'dT_dy']

sst_obser = data['sst_obser'][:, :64, :48]  # (9861, 41, 201)  #
print(sst_obser.shape)  # (9861, 41, 201)
slfh = data['slfh'][:, :64, :48]  # (9861, 41, 201)
sshf = data['sshf'][:, :64, :48]  # (9861, 41, 201)
ssr = data['ssr'][:, :64, :48]  # (9861, 41, 201)   # 可惜这个是 0
str = data['str'][:, :64, :48]  # (9861, 41, 201)
mld = data['mld'][:, :64, :48]  # (9861, 41, 201)
analysis_temp = data['analysis_temp'][:, :64, :48]  # (9861, 41, 201)
u = data['u'][:, :64, :48]  # (9861, 41, 201)
v = data['v'][:, :64, :48]  # (9861, 41, 201)
T_d = data['T_d'][:, :64, :48]  # (9861, 41, 201)
u_d = data['u_d'][:, :64, :48]  # (9861, 41, 201)
v_d = data['v_d'][:, :64, :48]  # (9861, 41, 201)
xx = data['xx'][:, :64, :48]  # (9861, 41, 201)
yy = data['yy'][:, :64, :48]  # (9861, 41, 201)
print(np.min(T_d))
print(np.max(T_d))
# print(xx)  # -5 -4.75 ... 4.75
# print(yy)  # 190  190.25 ... 239.75

dT_dt = data['dT_dt'][:, :64, :48]
dT_dx = data['dT_dx'][:, :64, :48]
dT_dy = data['dT_dy'][:, :64, :48]

['slfh', 'sshf', 'ssr', 'str', 'sst_obser', 'T_d', 'u_d', 'v_d', 'mld', 'analysis_temp', 'u', 'v', 'dT_dt', 'xx', 'yy', 'dT_dx', 'dT_dy']
(9861, 64, 48)
23.716982
37.334187


In [3]:
Q_net = (slfh + sshf + ssr + str)
Q_net.shape
Q_net = Q_net / (1025 * 4000 * mld)

In [4]:
ZAdv = u * dT_dx
ZAdv.shape

(9861, 64, 48)

In [5]:
MAdv = v * dT_dy
MAdv.shape

(9861, 64, 48)

In [6]:
VAdv = v_d * ((sst_obser -T_d ) / mld)
VAdv.shape

(9861, 64, 48)

In [7]:
R_origin = dT_dt - (Q_net[:9860,:,:] + ZAdv[:9860,:,:] + MAdv[:9860,:,:] + VAdv[:9860,:,:])
R_origin.shape

(9860, 64, 48)

In [8]:
R_origin

array([[[ 3.26215172e-02,  5.79827408e-02,  6.30745573e-02, ...,
         -1.78314650e-01, -1.91217148e-01, -2.26659268e-01],
        [-6.36579166e-02, -4.71776067e-02, -7.29218859e-02, ...,
         -2.34951265e-01, -2.74865411e-01, -2.38458516e-01],
        [-2.94864985e-01, -2.99270891e-01, -2.61218485e-01, ...,
         -2.76823026e-01, -3.04430484e-01, -2.88657794e-01],
        ...,
        [-2.55862385e-01, -2.08802831e-01, -1.67099721e-01, ...,
         -7.78051367e-02, -8.51047794e-02, -8.43447865e-02],
        [-2.40018355e-01, -1.78716144e-01, -1.42567560e-01, ...,
         -6.02849191e-02, -6.15976725e-02, -5.98325911e-02],
        [-1.85115216e-01, -1.42693628e-01, -1.96896772e-01, ...,
         -7.49336817e-02, -6.99024426e-02, -6.78678301e-02]],

       [[-3.27055120e-02, -3.76307019e-02, -7.50344916e-02, ...,
         -2.00272676e-02,  2.81071510e-03,  4.05671735e-02],
        [-1.54167501e-01, -2.18736886e-01, -2.11292114e-01, ...,
         -7.27621272e-02, -3.87034807e

In [9]:
"""我们方法修正项"""

'我们方法修正项'

In [10]:
data1 = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\Dt_with_R_e1.npz')
data2 = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\Dt_with_R_e2.npz')
data3 = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\Dt_with_R_e3.npz')
data4 = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\Dt_with_R_e4.npz')
data5 = np.load(r'D:\heat_wave\WEIO\expand_WEIO\last\Dt_with_R_e5.npz')
data1.files

['dt_R']

In [11]:
dt_R1 = data1['dt_R'][:].reshape(9860,64,48)
dt_R2 = data2['dt_R'][:].reshape(9860,64,48)
dt_R3 = data3['dt_R'][:].reshape(9860,64,48)
dt_R4 = data4['dt_R'][:].reshape(9860,64,48)
dt_R5 = data5['dt_R'][:].reshape(9860,64,48)
dt_R1.shape

(9860, 64, 48)

In [12]:
"修正后的R"
R_recur1 = dt_R1 - dT_dt
R_recur2 = dt_R2 - dT_dt
R_recur3 = dt_R3 - dT_dt
R_recur4 = dt_R4 - dT_dt
R_recur5 = dt_R5 - dT_dt
R_recur1

array([[[ 1.73735432e-02,  4.85667214e-03, -2.55575068e-02, ...,
          1.96566105e-01,  2.02255100e-01,  2.36916840e-01],
        [ 8.74748826e-02,  6.37643114e-02,  4.91898581e-02, ...,
          2.38168091e-01,  2.37328544e-01,  2.44811326e-01],
        [ 1.65114850e-01,  1.67971238e-01,  1.26112044e-01, ...,
          2.51074195e-01,  2.62383312e-01,  2.78906912e-01],
        ...,
        [ 1.09518662e-01,  9.80079770e-02,  6.68130666e-02, ...,
          8.48220736e-02,  7.99809098e-02,  8.79199952e-02],
        [ 8.68300945e-02,  7.68781155e-02,  6.92745000e-02, ...,
          8.32846314e-02,  8.19590688e-02,  8.15529674e-02],
        [ 6.42703772e-02,  5.75827807e-02,  8.39595944e-02, ...,
          8.84873793e-02,  9.38740373e-02,  1.08171299e-01]],

       [[ 8.56851228e-03,  3.69033106e-02,  4.11709473e-02, ...,
          2.59865001e-02,  1.43084303e-03, -5.11436164e-03],
        [ 1.01266757e-01,  1.27773792e-01,  1.16038717e-01, ...,
          5.82977347e-02,  3.40682790e

In [13]:
R_recur = ((R_recur1 + R_recur2 + R_recur3 + R_recur4 + R_recur5) / 5)
R_recur

array([[[ 0.02545035,  0.00163376, -0.02917834, ...,  0.1926109 ,
          0.19982931,  0.22878826],
        [ 0.09540649,  0.06012198,  0.04407824, ...,  0.23339815,
          0.23424546,  0.23700686],
        [ 0.16967328,  0.1647869 ,  0.11940988, ...,  0.2460546 ,
          0.2585153 ,  0.27059644],
        ...,
        [ 0.11314262,  0.09568087,  0.06067975, ...,  0.08031805,
          0.07686941,  0.07964358],
        [ 0.09004607,  0.07513298,  0.0633416 , ...,  0.07924847,
          0.07974195,  0.07338466],
        [ 0.06858536,  0.05494875,  0.07718669, ...,  0.08430934,
          0.0921771 ,  0.10006163]],

       [[ 0.01504593,  0.03393155,  0.03595721, ...,  0.02163858,
         -0.00048031, -0.01307513],
        [ 0.10796496,  0.12393083,  0.10995052, ...,  0.05373283,
          0.03156283,  0.00981142],
        [ 0.16661993,  0.15988052,  0.1413393 , ...,  0.06300773,
          0.02318634,  0.0085989 ],
        ...,
        [ 0.12030299,  0.10869499,  0.09239998, ..., -

In [14]:
# 首先 我们先画 DJF季节的贡献图   DJF： 12  1  2 月  即  12/01 - 02/31  当然，会做一个区域平均
# 同时，我们也会对每一年的DJF做一个平均  ---》1993 - 2019年

In [67]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2005,3,1)) 
index

4442

In [36]:
# DJF   93
Q_net_DJF_93 = Q_net[:59,:,:]
ZAdv_DJF_93 = ZAdv[:59,:,:]
MAdv_DJF_93 = MAdv[:59,:,:]
VAdv_DJF_93 = VAdv[:59,:,:]
R_origin_DJF_93 = R_origin[:59,:,:]
dT_dt_DJF_93 = dT_dt[:59,:,:]
R_recur_DJF_93 = R_recur[:59,:,:]

In [38]:
# DJF   94
Q_net_DJF_94 = Q_net[334:424,:,:]
ZAdv_DJF_94 = ZAdv[334:424,:,:]
MAdv_DJF_94 = MAdv[334:424,:,:]
VAdv_DJF_94 = VAdv[334:424,:,:]
R_origin_DJF_94 = R_origin[334:424,:,:]
dT_dt_DJF_94 = dT_dt[334:424,:,:]
R_recur_DJF_94 = R_recur[334:424,:,:]

In [40]:
# DJF   95
Q_net_DJF_95 = Q_net[699:789,:,:]
ZAdv_DJF_95 = ZAdv[699:789,:,:]
MAdv_DJF_95 = MAdv[699:789,:,:]
VAdv_DJF_95 = VAdv[699:789,:,:]
R_origin_DJF_95 = R_origin[699:789,:,:]
dT_dt_DJF_95 = dT_dt[699:789,:,:]
R_recur_DJF_95 = R_recur[699:789,:,:]

In [34]:
# DJF   96
Q_net_DJF_96 = Q_net[1064:1155,:,:]
ZAdv_DJF_96 = ZAdv[1064:1155,:,:]
MAdv_DJF_96 = MAdv[1064:1155,:,:]
VAdv_DJF_96 = VAdv[1064:1155,:,:]
R_origin_DJF_96 = R_origin[1064:1155,:,:]
dT_dt_DJF_96 = dT_dt[1064:1155,:,:]
R_recur_DJF_96 = R_recur[1064:1155,:,:]

In [44]:
# DJF   97
Q_net_DJF_97 = Q_net[1430:1520,:,:]
ZAdv_DJF_97 = ZAdv[1430:1520,:,:]
MAdv_DJF_97 = MAdv[1430:1520,:,:]
VAdv_DJF_97 = VAdv[1430:1520,:,:]
R_origin_DJF_97 = R_origin[1430:1520,:,:]
dT_dt_DJF_97 = dT_dt[1430:1520,:,:]
R_recur_DJF_97 = R_recur[1430:1520,:,:]

In [47]:
# DJF   98
Q_net_DJF_98 = Q_net[1795:1885,:,:]
ZAdv_DJF_98 = ZAdv[1795:1885,:,:]
MAdv_DJF_98 = MAdv[1795:1885,:,:]
VAdv_DJF_98 = VAdv[1795:1885,:,:]
R_origin_DJF_98 = R_origin[1795:1885,:,:]
dT_dt_DJF_98 = dT_dt[1795:1885,:,:]
R_recur_DJF_98 = R_recur[1795:1885,:,:]

In [50]:
# DJF   99
Q_net_DJF_99 = Q_net[2160:2250,:,:]
ZAdv_DJF_99 = ZAdv[2160:2250,:,:]
MAdv_DJF_99 = MAdv[2160:2250,:,:]
VAdv_DJF_99 = VAdv[2160:2250,:,:]
R_origin_DJF_99 = R_origin[2160:2250,:,:]
dT_dt_DJF_99 = dT_dt[2160:2250,:,:]
R_recur_DJF_99 = R_recur[2160:2250,:,:]

In [53]:
# DJF   00
Q_net_DJF_00 = Q_net[2525:2616,:,:]
ZAdv_DJF_00 = ZAdv[2525:2616,:,:]
MAdv_DJF_00 = MAdv[2525:2616,:,:]
VAdv_DJF_00 = VAdv[2525:2616,:,:]
R_origin_DJF_00 = R_origin[2525:2616,:,:]
dT_dt_DJF_00 = dT_dt[2525:2616,:,:]
R_recur_DJF_00 = R_recur[2525:2616,:,:]

In [56]:
# DJF   01
Q_net_DJF_01 = Q_net[2891:2981,:,:]
ZAdv_DJF_01 = ZAdv[2891:2981,:,:]
MAdv_DJF_01 = MAdv[2891:2981,:,:]
VAdv_DJF_01 = VAdv[2891:2981,:,:]
R_origin_DJF_01 = R_origin[2891:2981,:,:]
dT_dt_DJF_01 = dT_dt[2891:2981,:,:]
R_recur_DJF_01 = R_recur[2891:2981,:,:]

In [59]:
# DJF   02
Q_net_DJF_02 = Q_net[3256:3346,:,:]
ZAdv_DJF_02 = ZAdv[3256:3346,:,:]
MAdv_DJF_02 = MAdv[3256:3346,:,:]
VAdv_DJF_02 = VAdv[3256:3346,:,:]
R_origin_DJF_02 = R_origin[3256:3346,:,:]
dT_dt_DJF_02 = dT_dt[3256:3346,:,:]
R_recur_DJF_02 = R_recur[3256:3346,:,:]

In [62]:
# DJF   03
Q_net_DJF_03 = Q_net[3621:3711,:,:]
ZAdv_DJF_03 = ZAdv[3621:3711,:,:]
MAdv_DJF_03 = MAdv[3621:3711,:,:]
VAdv_DJF_03 = VAdv[3621:3711,:,:]
R_origin_DJF_03 = R_origin[3621:3711,:,:]
dT_dt_DJF_03 = dT_dt[3621:3711,:,:]
R_recur_DJF_03 = R_recur[3621:3711,:,:]

In [65]:
# DJF   04
Q_net_DJF_04 = Q_net[3986:4077,:,:]
ZAdv_DJF_04 = ZAdv[3986:4077,:,:]
MAdv_DJF_04 = MAdv[3986:4077,:,:]
VAdv_DJF_04 = VAdv[3986:4077,:,:]
R_origin_DJF_04 = R_origin[3986:4077,:,:]
dT_dt_DJF_04 = dT_dt[3986:4077,:,:]
R_recur_DJF_04 = R_recur[3986:4077,:,:]

In [68]:
# DJF   05
Q_net_DJF_05 = Q_net[4352:4442,:,:]
ZAdv_DJF_05 = ZAdv[4352:4442,:,:]
MAdv_DJF_05 = MAdv[4352:4442,:,:]
VAdv_DJF_05 = VAdv[4352:4442,:,:]
R_origin_DJF_05 = R_origin[4352:4442,:,:]
dT_dt_DJF_05 = dT_dt[4352:4442,:,:]
R_recur_DJF_05 = R_recur[4352:4442,:,:]

In [72]:
# DJF   06
Q_net_DJF_06 = Q_net[4717:4807,:,:]
ZAdv_DJF_06 = ZAdv[4717:4807,:,:]
MAdv_DJF_06 = MAdv[4717:4807,:,:]
VAdv_DJF_06 = VAdv[4717:4807,:,:]
R_origin_DJF_06 = R_origin[4717:4807,:,:]
dT_dt_DJF_06 = dT_dt[4717:4807,:,:]
R_recur_DJF_06 = R_recur[4717:4807,:,:]

In [75]:
# DJF   07
Q_net_DJF_07 = Q_net[5082:5172,:,:]
ZAdv_DJF_07 = ZAdv[5082:5172,:,:]
MAdv_DJF_07 = MAdv[5082:5172,:,:]
VAdv_DJF_07 = VAdv[5082:5172,:,:]
R_origin_DJF_07 = R_origin[5082:5172,:,:]
dT_dt_DJF_07 = dT_dt[5082:5172,:,:]
R_recur_DJF_07 = R_recur[5082:5172,:,:]

In [78]:
# DJF   08
Q_net_DJF_08 = Q_net[5447:5538,:,:]
ZAdv_DJF_08 = ZAdv[5447:5538,:,:]
MAdv_DJF_08 = MAdv[5447:5538,:,:]
VAdv_DJF_08 = VAdv[5447:5538,:,:]
R_origin_DJF_08 = R_origin[5447:5538,:,:]
dT_dt_DJF_08 = dT_dt[5447:5538,:,:]
R_recur_DJF_08 = R_recur[5447:5538,:,:]

In [81]:
# DJF   09
Q_net_DJF_09 = Q_net[5813:5903,:,:]
ZAdv_DJF_09 = ZAdv[5813:5903,:,:]
MAdv_DJF_09 = MAdv[5813:5903,:,:]
VAdv_DJF_09 = VAdv[5813:5903,:,:]
R_origin_DJF_09 = R_origin[5813:5903,:,:]
dT_dt_DJF_09 = dT_dt[5813:5903,:,:]
R_recur_DJF_09 = R_recur[5813:5903,:,:]

In [84]:
# DJF   10
Q_net_DJF_10 = Q_net[6178:6268,:,:]
ZAdv_DJF_10 = ZAdv[6178:6268,:,:]
MAdv_DJF_10 = MAdv[6178:6268,:,:]
VAdv_DJF_10 = VAdv[6178:6268,:,:]
R_origin_DJF_10 = R_origin[6178:6268,:,:]
dT_dt_DJF_10 = dT_dt[6178:6268,:,:]
R_recur_DJF_10 = R_recur[6178:6268,:,:]

In [88]:
# DJF   11
Q_net_DJF_11 = Q_net[6543:6633,:,:]
ZAdv_DJF_11 = ZAdv[6543:6633,:,:]
MAdv_DJF_11 = MAdv[6543:6633,:,:]
VAdv_DJF_11 = VAdv[6543:6633,:,:]
R_origin_DJF_11 = R_origin[6543:6633,:,:]
dT_dt_DJF_11 = dT_dt[6543:6633,:,:]
R_recur_DJF_11 = R_recur[6543:6633,:,:]

In [91]:
# DJF   12
Q_net_DJF_12 = Q_net[6908:6999,:,:]
ZAdv_DJF_12 = ZAdv[6908:6999,:,:]
MAdv_DJF_12 = MAdv[6908:6999,:,:]
VAdv_DJF_12 = VAdv[6908:6999,:,:]
R_origin_DJF_12 = R_origin[6908:6999,:,:]
dT_dt_DJF_12 = dT_dt[6908:6999,:,:]
R_recur_DJF_12 = R_recur[6908:6999,:,:]

In [94]:
# DJF   13
Q_net_DJF_13 = Q_net[7274:7364,:,:]
ZAdv_DJF_13 = ZAdv[7274:7364,:,:]
MAdv_DJF_13 = MAdv[7274:7364,:,:]
VAdv_DJF_13 = VAdv[7274:7364,:,:]
R_origin_DJF_13 = R_origin[7274:7364,:,:]
dT_dt_DJF_13 = dT_dt[7274:7364,:,:]
R_recur_DJF_13 = R_recur[7274:7364,:,:]

In [97]:
# DJF   14
Q_net_DJF_14 = Q_net[7639:7729,:,:]
ZAdv_DJF_14 = ZAdv[7639:7729,:,:]
MAdv_DJF_14 = MAdv[7639:7729,:,:]
VAdv_DJF_14 = VAdv[7639:7729,:,:]
R_origin_DJF_14 = R_origin[7639:7729,:,:]
dT_dt_DJF_14 = dT_dt[7639:7729,:,:]
R_recur_DJF_14 = R_recur[7639:7729,:,:]

In [100]:
# DJF   15
Q_net_DJF_15 = Q_net[8004:8094,:,:]
ZAdv_DJF_15 = ZAdv[8004:8094,:,:]
MAdv_DJF_15 = MAdv[8004:8094,:,:]
VAdv_DJF_15 = VAdv[8004:8094,:,:]
R_origin_DJF_15 = R_origin[8004:8094,:,:]
dT_dt_DJF_15 = dT_dt[8004:8094,:,:]
R_recur_DJF_15 = R_recur[8004:8094,:,:]

In [103]:
# DJF   16
Q_net_DJF_16 = Q_net[8369:8460,:,:]
ZAdv_DJF_16 = ZAdv[8369:8460,:,:]
MAdv_DJF_16 = MAdv[8369:8460,:,:]
VAdv_DJF_16 = VAdv[8369:8460,:,:]
R_origin_DJF_16 = R_origin[8369:8460,:,:]
dT_dt_DJF_16 = dT_dt[8369:8460,:,:]
R_recur_DJF_16 = R_recur[8369:8460,:,:]

In [106]:
# DJF   17
Q_net_DJF_17 = Q_net[8735:8825,:,:]
ZAdv_DJF_17 = ZAdv[8735:8825,:,:]
MAdv_DJF_17 = MAdv[8735:8825,:,:]
VAdv_DJF_17 = VAdv[8735:8825,:,:]
R_origin_DJF_17 = R_origin[8735:8825,:,:]
dT_dt_DJF_17 = dT_dt[8735:8825,:,:]
R_recur_DJF_17 = R_recur[8735:8825,:,:]

In [109]:
# DJF   18
Q_net_DJF_18 = Q_net[9100:9190,:,:]
ZAdv_DJF_18 = ZAdv[9100:9190,:,:]
MAdv_DJF_18 = MAdv[9100:9190,:,:]
VAdv_DJF_18 = VAdv[9100:9190,:,:]
R_origin_DJF_18 = R_origin[9100:9190,:,:]
dT_dt_DJF_18 = dT_dt[9100:9190,:,:]
R_recur_DJF_18 = R_recur[9100:9190,:,:]

In [112]:
# DJF   19
Q_net_DJF_19 = Q_net[9465:9555,:,:]
ZAdv_DJF_19 = ZAdv[9465:9555,:,:]
MAdv_DJF_19 = MAdv[9465:9555,:,:]
VAdv_DJF_19 = VAdv[9465:9555,:,:]
R_origin_DJF_19 = R_origin[9465:9555,:,:]
dT_dt_DJF_19 = dT_dt[9465:9555,:,:]
R_recur_DJF_19 = R_recur[9465:9555,:,:]

In [114]:
Q_net_DJF_93 = np.mean(Q_net_DJF_93, axis = (1,2)).reshape(-1,1)
Q_net_DJF_94 = np.mean(Q_net_DJF_94, axis = (1,2)).reshape(-1,1)
Q_net_DJF_95 = np.mean(Q_net_DJF_95, axis = (1,2)).reshape(-1,1)
Q_net_DJF_96 = np.mean(Q_net_DJF_96, axis = (1,2)).reshape(-1,1)
Q_net_DJF_97 = np.mean(Q_net_DJF_97, axis = (1,2)).reshape(-1,1)
Q_net_DJF_98 = np.mean(Q_net_DJF_98, axis = (1,2)).reshape(-1,1)
Q_net_DJF_99 = np.mean(Q_net_DJF_99, axis = (1,2)).reshape(-1,1)
Q_net_DJF_00 = np.mean(Q_net_DJF_00, axis = (1,2)).reshape(-1,1)
Q_net_DJF_01 = np.mean(Q_net_DJF_01, axis = (1,2)).reshape(-1,1)
Q_net_DJF_02 = np.mean(Q_net_DJF_02, axis = (1,2)).reshape(-1,1)
Q_net_DJF_03 = np.mean(Q_net_DJF_03, axis = (1,2)).reshape(-1,1)
Q_net_DJF_04 = np.mean(Q_net_DJF_04, axis = (1,2)).reshape(-1,1)
Q_net_DJF_05 = np.mean(Q_net_DJF_05, axis = (1,2)).reshape(-1,1)
Q_net_DJF_06 = np.mean(Q_net_DJF_06, axis = (1,2)).reshape(-1,1)
Q_net_DJF_07 = np.mean(Q_net_DJF_07,axis = (1,2)).reshape(-1,1)
Q_net_DJF_08 = np.mean(Q_net_DJF_08, axis = (1,2)).reshape(-1,1)
Q_net_DJF_09 = np.mean(Q_net_DJF_09, axis = (1,2)).reshape(-1,1)
Q_net_DJF_10 = np.mean(Q_net_DJF_10, axis = (1,2)).reshape(-1,1)
Q_net_DJF_11 = np.mean(Q_net_DJF_11, axis = (1,2)).reshape(-1,1)
Q_net_DJF_12 = np.mean(Q_net_DJF_12, axis = (1,2)).reshape(-1,1)
Q_net_DJF_13 = np.mean(Q_net_DJF_13, axis = (1,2)).reshape(-1,1)
Q_net_DJF_14 = np.mean(Q_net_DJF_14, axis = (1,2)).reshape(-1,1)
Q_net_DJF_15 = np.mean(Q_net_DJF_15, axis = (1,2)).reshape(-1,1)
Q_net_DJF_16 = np.mean(Q_net_DJF_16, axis = (1,2)).reshape(-1,1)
Q_net_DJF_17 = np.mean(Q_net_DJF_17, axis = (1,2)).reshape(-1,1)
Q_net_DJF_18 = np.mean(Q_net_DJF_18, axis = (1,2)).reshape(-1,1)
Q_net_DJF_19 = np.mean(Q_net_DJF_19, axis = (1,2)).reshape(-1,1)

AxisError: axis 2 is out of bounds for array of dimension 2

In [115]:
Q_net_DJF_93 = np.mean(Q_net_DJF_93).reshape(-1,1)
Q_net_DJF_94 = np.mean(Q_net_DJF_94).reshape(-1,1)
Q_net_DJF_95 = np.mean(Q_net_DJF_95).reshape(-1,1)
Q_net_DJF_96 = np.mean(Q_net_DJF_96).reshape(-1,1)
Q_net_DJF_97 = np.mean(Q_net_DJF_97).reshape(-1,1)
Q_net_DJF_98 = np.mean(Q_net_DJF_98).reshape(-1,1)
Q_net_DJF_99 = np.mean(Q_net_DJF_99).reshape(-1,1)
Q_net_DJF_00 = np.mean(Q_net_DJF_00).reshape(-1,1)
Q_net_DJF_01 = np.mean(Q_net_DJF_01).reshape(-1,1)
Q_net_DJF_02 = np.mean(Q_net_DJF_02).reshape(-1,1)
Q_net_DJF_03 = np.mean(Q_net_DJF_03).reshape(-1,1)
Q_net_DJF_04 = np.mean(Q_net_DJF_04).reshape(-1,1)
Q_net_DJF_05 = np.mean(Q_net_DJF_05).reshape(-1,1)
Q_net_DJF_06 = np.mean(Q_net_DJF_06).reshape(-1,1)
Q_net_DJF_07 = np.mean(Q_net_DJF_07).reshape(-1,1)
Q_net_DJF_08 = np.mean(Q_net_DJF_08).reshape(-1,1)
Q_net_DJF_09 = np.mean(Q_net_DJF_09).reshape(-1,1)
Q_net_DJF_10 = np.mean(Q_net_DJF_10).reshape(-1,1)
Q_net_DJF_11 = np.mean(Q_net_DJF_11).reshape(-1,1)
Q_net_DJF_12 = np.mean(Q_net_DJF_12).reshape(-1,1)
Q_net_DJF_13 = np.mean(Q_net_DJF_13).reshape(-1,1)
Q_net_DJF_14 = np.mean(Q_net_DJF_14).reshape(-1,1)
Q_net_DJF_15 = np.mean(Q_net_DJF_15).reshape(-1,1)
Q_net_DJF_16 = np.mean(Q_net_DJF_16).reshape(-1,1)
Q_net_DJF_17 = np.mean(Q_net_DJF_17).reshape(-1,1)
Q_net_DJF_18 = np.mean(Q_net_DJF_18).reshape(-1,1)
Q_net_DJF_19 = np.mean(Q_net_DJF_19).reshape(-1,1)

In [116]:
ZAdv_DJF_93 = np.mean(ZAdv_DJF_93, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_94 = np.mean(ZAdv_DJF_94, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_95 = np.mean(ZAdv_DJF_95, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_96 = np.mean(ZAdv_DJF_96, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_97 = np.mean(ZAdv_DJF_97, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_98 = np.mean(ZAdv_DJF_98, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_99 = np.mean(ZAdv_DJF_99, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_00 = np.mean(ZAdv_DJF_00, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_01 = np.mean(ZAdv_DJF_01, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_02 = np.mean(ZAdv_DJF_02, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_03 = np.mean(ZAdv_DJF_03, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_04 = np.mean(ZAdv_DJF_04, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_05 = np.mean(ZAdv_DJF_05, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_06 = np.mean(ZAdv_DJF_06, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_07 = np.mean(ZAdv_DJF_07, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_08 = np.mean(ZAdv_DJF_08, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_09 = np.mean(ZAdv_DJF_09, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_10 = np.mean(ZAdv_DJF_10, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_11 = np.mean(ZAdv_DJF_11, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_12 = np.mean(ZAdv_DJF_12, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_13 = np.mean(ZAdv_DJF_13, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_14 = np.mean(ZAdv_DJF_14, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_15 = np.mean(ZAdv_DJF_15, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_16 = np.mean(ZAdv_DJF_16, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_17 = np.mean(ZAdv_DJF_17, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_18 = np.mean(ZAdv_DJF_18, axis = (1,2)).reshape(-1,1)
ZAdv_DJF_19 = np.mean(ZAdv_DJF_19, axis = (1,2)).reshape(-1,1)

In [117]:
ZAdv_DJF_93 = np.mean(ZAdv_DJF_93).reshape(-1,1)
ZAdv_DJF_94 = np.mean(ZAdv_DJF_94).reshape(-1,1)
ZAdv_DJF_95 = np.mean(ZAdv_DJF_95).reshape(-1,1)
ZAdv_DJF_96 = np.mean(ZAdv_DJF_96).reshape(-1,1)
ZAdv_DJF_97 = np.mean(ZAdv_DJF_97).reshape(-1,1)
ZAdv_DJF_98 = np.mean(ZAdv_DJF_98).reshape(-1,1)
ZAdv_DJF_99 = np.mean(ZAdv_DJF_99).reshape(-1,1)
ZAdv_DJF_00 = np.mean(ZAdv_DJF_00).reshape(-1,1)
ZAdv_DJF_01 = np.mean(ZAdv_DJF_01).reshape(-1,1)
ZAdv_DJF_02 = np.mean(ZAdv_DJF_02).reshape(-1,1)
ZAdv_DJF_03 = np.mean(ZAdv_DJF_03).reshape(-1,1)
ZAdv_DJF_04 = np.mean(ZAdv_DJF_04).reshape(-1,1)
ZAdv_DJF_05 = np.mean(ZAdv_DJF_05).reshape(-1,1)
ZAdv_DJF_06 = np.mean(ZAdv_DJF_06).reshape(-1,1)
ZAdv_DJF_07 = np.mean(ZAdv_DJF_07).reshape(-1,1)
ZAdv_DJF_08 = np.mean(ZAdv_DJF_08).reshape(-1,1)
ZAdv_DJF_09 = np.mean(ZAdv_DJF_09).reshape(-1,1)
ZAdv_DJF_10 = np.mean(ZAdv_DJF_10).reshape(-1,1)
ZAdv_DJF_11 = np.mean(ZAdv_DJF_11).reshape(-1,1)
ZAdv_DJF_12 = np.mean(ZAdv_DJF_12).reshape(-1,1)
ZAdv_DJF_13 = np.mean(ZAdv_DJF_13).reshape(-1,1)
ZAdv_DJF_14 = np.mean(ZAdv_DJF_14).reshape(-1,1)
ZAdv_DJF_15 = np.mean(ZAdv_DJF_15).reshape(-1,1)
ZAdv_DJF_16 = np.mean(ZAdv_DJF_16).reshape(-1,1)
ZAdv_DJF_17 = np.mean(ZAdv_DJF_17).reshape(-1,1)
ZAdv_DJF_18 = np.mean(ZAdv_DJF_18).reshape(-1,1)
ZAdv_DJF_19 = np.mean(ZAdv_DJF_19).reshape(-1,1)

In [118]:
MAdv_DJF_93 = np.mean(MAdv_DJF_93, axis = (1,2)).reshape(-1,1)
MAdv_DJF_94 = np.mean(MAdv_DJF_94, axis = (1,2)).reshape(-1,1)
MAdv_DJF_95 = np.mean(MAdv_DJF_95, axis = (1,2)).reshape(-1,1)
MAdv_DJF_96 = np.mean(MAdv_DJF_96, axis = (1,2)).reshape(-1,1)
MAdv_DJF_97 = np.mean(MAdv_DJF_97, axis = (1,2)).reshape(-1,1)
MAdv_DJF_98 = np.mean(MAdv_DJF_98, axis = (1,2)).reshape(-1,1)
MAdv_DJF_99 = np.mean(MAdv_DJF_99, axis = (1,2)).reshape(-1,1)
MAdv_DJF_00 = np.mean(MAdv_DJF_00, axis = (1,2)).reshape(-1,1)
MAdv_DJF_01 = np.mean(MAdv_DJF_01, axis = (1,2)).reshape(-1,1)
MAdv_DJF_02 = np.mean(MAdv_DJF_02, axis = (1,2)).reshape(-1,1)
MAdv_DJF_03 = np.mean(MAdv_DJF_03, axis = (1,2)).reshape(-1,1)
MAdv_DJF_04 = np.mean(MAdv_DJF_04, axis = (1,2)).reshape(-1,1)
MAdv_DJF_05 = np.mean(MAdv_DJF_05, axis = (1,2)).reshape(-1,1)
MAdv_DJF_06 = np.mean(MAdv_DJF_06, axis = (1,2)).reshape(-1,1)
MAdv_DJF_07 = np.mean(MAdv_DJF_07, axis = (1,2)).reshape(-1,1)
MAdv_DJF_08 = np.mean(MAdv_DJF_08, axis = (1,2)).reshape(-1,1)
MAdv_DJF_09 = np.mean(MAdv_DJF_09, axis = (1,2)).reshape(-1,1)
MAdv_DJF_10 = np.mean(MAdv_DJF_10, axis = (1,2)).reshape(-1,1)
MAdv_DJF_11 = np.mean(MAdv_DJF_11, axis = (1,2)).reshape(-1,1)
MAdv_DJF_12 = np.mean(MAdv_DJF_12, axis = (1,2)).reshape(-1,1)
MAdv_DJF_13 = np.mean(MAdv_DJF_13, axis = (1,2)).reshape(-1,1)
MAdv_DJF_14 = np.mean(MAdv_DJF_14, axis = (1,2)).reshape(-1,1)
MAdv_DJF_15 = np.mean(MAdv_DJF_15, axis = (1,2)).reshape(-1,1)
MAdv_DJF_16 = np.mean(MAdv_DJF_16, axis = (1,2)).reshape(-1,1)
MAdv_DJF_17 = np.mean(MAdv_DJF_17, axis = (1,2)).reshape(-1,1)
MAdv_DJF_18 = np.mean(MAdv_DJF_18, axis = (1,2)).reshape(-1,1)
MAdv_DJF_19 = np.mean(MAdv_DJF_19, axis = (1,2)).reshape(-1,1)

In [120]:
MAdv_DJF_93 = np.mean(MAdv_DJF_93).reshape(-1,1)
MAdv_DJF_94 = np.mean(MAdv_DJF_94).reshape(-1,1)
MAdv_DJF_95 = np.mean(MAdv_DJF_95).reshape(-1,1)
MAdv_DJF_96 = np.mean(MAdv_DJF_96).reshape(-1,1)
MAdv_DJF_97 = np.mean(MAdv_DJF_97).reshape(-1,1)
MAdv_DJF_98 = np.mean(MAdv_DJF_98).reshape(-1,1)
MAdv_DJF_99 = np.mean(MAdv_DJF_99).reshape(-1,1)
MAdv_DJF_00 = np.mean(MAdv_DJF_00).reshape(-1,1)
MAdv_DJF_01 = np.mean(MAdv_DJF_01).reshape(-1,1)
MAdv_DJF_02 = np.mean(MAdv_DJF_02).reshape(-1,1)
MAdv_DJF_03 = np.mean(MAdv_DJF_03).reshape(-1,1)
MAdv_DJF_04 = np.mean(MAdv_DJF_04).reshape(-1,1)
MAdv_DJF_05 = np.mean(MAdv_DJF_05).reshape(-1,1)
MAdv_DJF_06 = np.mean(MAdv_DJF_06).reshape(-1,1)
MAdv_DJF_07 = np.mean(MAdv_DJF_07).reshape(-1,1)
MAdv_DJF_08 = np.mean(MAdv_DJF_08).reshape(-1,1)
MAdv_DJF_09 = np.mean(MAdv_DJF_09).reshape(-1,1)
MAdv_DJF_10 = np.mean(MAdv_DJF_10).reshape(-1,1)
MAdv_DJF_11 = np.mean(MAdv_DJF_11).reshape(-1,1)
MAdv_DJF_12 = np.mean(MAdv_DJF_12).reshape(-1,1)
MAdv_DJF_13 = np.mean(MAdv_DJF_13).reshape(-1,1)
MAdv_DJF_14 = np.mean(MAdv_DJF_14).reshape(-1,1)
MAdv_DJF_15 = np.mean(MAdv_DJF_15).reshape(-1,1)
MAdv_DJF_16 = np.mean(MAdv_DJF_16).reshape(-1,1)
MAdv_DJF_17 = np.mean(MAdv_DJF_17).reshape(-1,1)
MAdv_DJF_18 = np.mean(MAdv_DJF_18).reshape(-1,1)
MAdv_DJF_19 = np.mean(MAdv_DJF_19).reshape(-1,1)

In [121]:
VAdv_DJF_93 = np.mean(VAdv_DJF_93, axis = (1,2)).reshape(-1,1)
VAdv_DJF_94 = np.mean(VAdv_DJF_94, axis = (1,2)).reshape(-1,1)
VAdv_DJF_95 = np.mean(VAdv_DJF_95, axis = (1,2)).reshape(-1,1)
VAdv_DJF_96 = np.mean(VAdv_DJF_96, axis = (1,2)).reshape(-1,1)
VAdv_DJF_97 = np.mean(VAdv_DJF_97, axis = (1,2)).reshape(-1,1)
VAdv_DJF_98 = np.mean(VAdv_DJF_98, axis = (1,2)).reshape(-1,1)
VAdv_DJF_99 = np.mean(VAdv_DJF_99, axis = (1,2)).reshape(-1,1)
VAdv_DJF_00 = np.mean(VAdv_DJF_00, axis = (1,2)).reshape(-1,1)
VAdv_DJF_01 = np.mean(VAdv_DJF_01, axis = (1,2)).reshape(-1,1)
VAdv_DJF_02 = np.mean(VAdv_DJF_02, axis = (1,2)).reshape(-1,1)
VAdv_DJF_03 = np.mean(VAdv_DJF_03, axis = (1,2)).reshape(-1,1)
VAdv_DJF_04 = np.mean(VAdv_DJF_04, axis = (1,2)).reshape(-1,1)
VAdv_DJF_05 = np.mean(VAdv_DJF_05, axis = (1,2)).reshape(-1,1)
VAdv_DJF_06 = np.mean(VAdv_DJF_06, axis = (1,2)).reshape(-1,1)
VAdv_DJF_07 = np.mean(VAdv_DJF_07, axis = (1,2)).reshape(-1,1)
VAdv_DJF_08 = np.mean(VAdv_DJF_08, axis = (1,2)).reshape(-1,1)
VAdv_DJF_09 = np.mean(VAdv_DJF_09, axis = (1,2)).reshape(-1,1)
VAdv_DJF_10 = np.mean(VAdv_DJF_10, axis = (1,2)).reshape(-1,1)
VAdv_DJF_11 = np.mean(VAdv_DJF_11, axis = (1,2)).reshape(-1,1)
VAdv_DJF_12 = np.mean(VAdv_DJF_12, axis = (1,2)).reshape(-1,1)
VAdv_DJF_13 = np.mean(VAdv_DJF_13, axis = (1,2)).reshape(-1,1)
VAdv_DJF_14 = np.mean(VAdv_DJF_14, axis = (1,2)).reshape(-1,1)
VAdv_DJF_15 = np.mean(VAdv_DJF_15, axis = (1,2)).reshape(-1,1)
VAdv_DJF_16 = np.mean(VAdv_DJF_16, axis = (1,2)).reshape(-1,1)
VAdv_DJF_17 = np.mean(VAdv_DJF_17, axis = (1,2)).reshape(-1,1)
VAdv_DJF_18 = np.mean(VAdv_DJF_18, axis = (1,2)).reshape(-1,1)
VAdv_DJF_19 = np.mean(VAdv_DJF_19, axis = (1,2)).reshape(-1,1)

In [122]:
VAdv_DJF_93 = np.mean(VAdv_DJF_93).reshape(-1,1)
VAdv_DJF_94 = np.mean(VAdv_DJF_94).reshape(-1,1)
VAdv_DJF_95 = np.mean(VAdv_DJF_95).reshape(-1,1)
VAdv_DJF_96 = np.mean(VAdv_DJF_96).reshape(-1,1)
VAdv_DJF_97 = np.mean(VAdv_DJF_97).reshape(-1,1)
VAdv_DJF_98 = np.mean(VAdv_DJF_98).reshape(-1,1)
VAdv_DJF_99 = np.mean(VAdv_DJF_99).reshape(-1,1)
VAdv_DJF_00 = np.mean(VAdv_DJF_00).reshape(-1,1)
VAdv_DJF_01 = np.mean(VAdv_DJF_01).reshape(-1,1)
VAdv_DJF_02 = np.mean(VAdv_DJF_02).reshape(-1,1)
VAdv_DJF_03 = np.mean(VAdv_DJF_03).reshape(-1,1)
VAdv_DJF_04 = np.mean(VAdv_DJF_04).reshape(-1,1)
VAdv_DJF_05 = np.mean(VAdv_DJF_05).reshape(-1,1)
VAdv_DJF_06 = np.mean(VAdv_DJF_06).reshape(-1,1)
VAdv_DJF_07 = np.mean(VAdv_DJF_07).reshape(-1,1)
VAdv_DJF_08 = np.mean(VAdv_DJF_08).reshape(-1,1)
VAdv_DJF_09 = np.mean(VAdv_DJF_09).reshape(-1,1)
VAdv_DJF_10 = np.mean(VAdv_DJF_10).reshape(-1,1)
VAdv_DJF_11 = np.mean(VAdv_DJF_11).reshape(-1,1)
VAdv_DJF_12 = np.mean(VAdv_DJF_12).reshape(-1,1)
VAdv_DJF_13 = np.mean(VAdv_DJF_13).reshape(-1,1)
VAdv_DJF_14 = np.mean(VAdv_DJF_14).reshape(-1,1)
VAdv_DJF_15 = np.mean(VAdv_DJF_15).reshape(-1,1)
VAdv_DJF_16 = np.mean(VAdv_DJF_16).reshape(-1,1)
VAdv_DJF_17 = np.mean(VAdv_DJF_17).reshape(-1,1)
VAdv_DJF_18 = np.mean(VAdv_DJF_18).reshape(-1,1)
VAdv_DJF_19 = np.mean(VAdv_DJF_19).reshape(-1,1)

In [123]:
R_origin_DJF_93 = np.mean(R_origin_DJF_93, axis = (1,2)).reshape(-1,1)
R_origin_DJF_94 = np.mean(R_origin_DJF_94, axis = (1,2)).reshape(-1,1)
R_origin_DJF_95 = np.mean(R_origin_DJF_95, axis = (1,2)).reshape(-1,1)
R_origin_DJF_96 = np.mean(R_origin_DJF_96, axis = (1,2)).reshape(-1,1)
R_origin_DJF_97 = np.mean(R_origin_DJF_97, axis = (1,2)).reshape(-1,1)
R_origin_DJF_98 = np.mean(R_origin_DJF_98, axis = (1,2)).reshape(-1,1)
R_origin_DJF_99 = np.mean(R_origin_DJF_99, axis = (1,2)).reshape(-1,1)
R_origin_DJF_00 = np.mean(R_origin_DJF_00, axis = (1,2)).reshape(-1,1)
R_origin_DJF_01 = np.mean(R_origin_DJF_01, axis = (1,2)).reshape(-1,1)
R_origin_DJF_02 = np.mean(R_origin_DJF_02, axis = (1,2)).reshape(-1,1)
R_origin_DJF_03 = np.mean(R_origin_DJF_03, axis = (1,2)).reshape(-1,1)
R_origin_DJF_04 = np.mean(R_origin_DJF_04, axis = (1,2)).reshape(-1,1)
R_origin_DJF_05 = np.mean(R_origin_DJF_05, axis = (1,2)).reshape(-1,1)
R_origin_DJF_06 = np.mean(R_origin_DJF_06, axis = (1,2)).reshape(-1,1)
R_origin_DJF_07 = np.mean(R_origin_DJF_07, axis = (1,2)).reshape(-1,1)
R_origin_DJF_08 = np.mean(R_origin_DJF_08, axis = (1,2)).reshape(-1,1)
R_origin_DJF_09 = np.mean(R_origin_DJF_09, axis = (1,2)).reshape(-1,1)
R_origin_DJF_10 = np.mean(R_origin_DJF_10, axis = (1,2)).reshape(-1,1)
R_origin_DJF_11 = np.mean(R_origin_DJF_11, axis = (1,2)).reshape(-1,1)
R_origin_DJF_12 = np.mean(R_origin_DJF_12, axis = (1,2)).reshape(-1,1)
R_origin_DJF_13 = np.mean(R_origin_DJF_13, axis = (1,2)).reshape(-1,1)
R_origin_DJF_14 = np.mean(R_origin_DJF_14, axis = (1,2)).reshape(-1,1)
R_origin_DJF_15 = np.mean(R_origin_DJF_15, axis = (1,2)).reshape(-1,1)
R_origin_DJF_16 = np.mean(R_origin_DJF_16, axis = (1,2)).reshape(-1,1)
R_origin_DJF_17 = np.mean(R_origin_DJF_17, axis = (1,2)).reshape(-1,1)
R_origin_DJF_18 = np.mean(R_origin_DJF_18, axis = (1,2)).reshape(-1,1)
R_origin_DJF_19 = np.mean(R_origin_DJF_19, axis = (1,2)).reshape(-1,1)

In [124]:
R_origin_DJF_93 = np.mean(R_origin_DJF_93).reshape(-1,1)
R_origin_DJF_94 = np.mean(R_origin_DJF_94).reshape(-1,1)
R_origin_DJF_95 = np.mean(R_origin_DJF_95).reshape(-1,1)
R_origin_DJF_96 = np.mean(R_origin_DJF_96).reshape(-1,1)
R_origin_DJF_97 = np.mean(R_origin_DJF_97).reshape(-1,1)
R_origin_DJF_98 = np.mean(R_origin_DJF_98).reshape(-1,1)
R_origin_DJF_99 = np.mean(R_origin_DJF_99).reshape(-1,1)
R_origin_DJF_00 = np.mean(R_origin_DJF_00).reshape(-1,1)
R_origin_DJF_01 = np.mean(R_origin_DJF_01).reshape(-1,1)
R_origin_DJF_02 = np.mean(R_origin_DJF_02).reshape(-1,1)
R_origin_DJF_03 = np.mean(R_origin_DJF_03).reshape(-1,1)
R_origin_DJF_04 = np.mean(R_origin_DJF_04).reshape(-1,1)
R_origin_DJF_05 = np.mean(R_origin_DJF_05).reshape(-1,1)
R_origin_DJF_06 = np.mean(R_origin_DJF_06).reshape(-1,1)
R_origin_DJF_07 = np.mean(R_origin_DJF_07).reshape(-1,1)
R_origin_DJF_08 = np.mean(R_origin_DJF_08).reshape(-1,1)
R_origin_DJF_09 = np.mean(R_origin_DJF_09).reshape(-1,1)
R_origin_DJF_10 = np.mean(R_origin_DJF_10).reshape(-1,1)
R_origin_DJF_11 = np.mean(R_origin_DJF_11).reshape(-1,1)
R_origin_DJF_12 = np.mean(R_origin_DJF_12).reshape(-1,1)
R_origin_DJF_13 = np.mean(R_origin_DJF_13).reshape(-1,1)
R_origin_DJF_14 = np.mean(R_origin_DJF_14).reshape(-1,1)
R_origin_DJF_15 = np.mean(R_origin_DJF_15).reshape(-1,1)
R_origin_DJF_16 = np.mean(R_origin_DJF_16).reshape(-1,1)
R_origin_DJF_17 = np.mean(R_origin_DJF_17).reshape(-1,1)
R_origin_DJF_18 = np.mean(R_origin_DJF_18).reshape(-1,1)
R_origin_DJF_19 = np.mean(R_origin_DJF_19).reshape(-1,1)

In [125]:
dT_dt_DJF_93 = np.mean(dT_dt_DJF_93, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_94 = np.mean(dT_dt_DJF_94, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_95 = np.mean(dT_dt_DJF_95, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_96 = np.mean(dT_dt_DJF_96, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_97 = np.mean(dT_dt_DJF_97, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_98 = np.mean(dT_dt_DJF_98, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_99 = np.mean(dT_dt_DJF_99, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_00 = np.mean(dT_dt_DJF_00, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_01 = np.mean(dT_dt_DJF_01, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_02 = np.mean(dT_dt_DJF_02, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_03 = np.mean(dT_dt_DJF_03, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_04 = np.mean(dT_dt_DJF_04, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_05 = np.mean(dT_dt_DJF_05, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_06 = np.mean(dT_dt_DJF_06, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_07 = np.mean(dT_dt_DJF_07, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_08 = np.mean(dT_dt_DJF_08, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_09 = np.mean(dT_dt_DJF_09, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_10 = np.mean(dT_dt_DJF_10, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_11 = np.mean(dT_dt_DJF_11, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_12 = np.mean(dT_dt_DJF_12, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_13 = np.mean(dT_dt_DJF_13, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_14 = np.mean(dT_dt_DJF_14, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_15 = np.mean(dT_dt_DJF_15, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_16 = np.mean(dT_dt_DJF_16, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_17 = np.mean(dT_dt_DJF_17, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_18 = np.mean(dT_dt_DJF_18, axis = (1,2)).reshape(-1,1)
dT_dt_DJF_19 = np.mean(dT_dt_DJF_19, axis = (1,2)).reshape(-1,1)

In [126]:
dT_dt_DJF_93 = np.mean(dT_dt_DJF_93).reshape(-1,1)
dT_dt_DJF_94 = np.mean(dT_dt_DJF_94).reshape(-1,1)
dT_dt_DJF_95 = np.mean(dT_dt_DJF_95).reshape(-1,1)
dT_dt_DJF_96 = np.mean(dT_dt_DJF_96).reshape(-1,1)
dT_dt_DJF_97 = np.mean(dT_dt_DJF_97).reshape(-1,1)
dT_dt_DJF_98 = np.mean(dT_dt_DJF_98).reshape(-1,1)
dT_dt_DJF_99 = np.mean(dT_dt_DJF_99).reshape(-1,1)
dT_dt_DJF_00 = np.mean(dT_dt_DJF_00).reshape(-1,1)
dT_dt_DJF_01 = np.mean(dT_dt_DJF_01).reshape(-1,1)
dT_dt_DJF_02 = np.mean(dT_dt_DJF_02).reshape(-1,1)
dT_dt_DJF_03 = np.mean(dT_dt_DJF_03).reshape(-1,1)
dT_dt_DJF_04 = np.mean(dT_dt_DJF_04).reshape(-1,1)
dT_dt_DJF_05 = np.mean(dT_dt_DJF_05).reshape(-1,1)
dT_dt_DJF_06 = np.mean(dT_dt_DJF_06).reshape(-1,1)
dT_dt_DJF_07 = np.mean(dT_dt_DJF_07).reshape(-1,1)
dT_dt_DJF_08 = np.mean(dT_dt_DJF_08).reshape(-1,1)
dT_dt_DJF_09 = np.mean(dT_dt_DJF_09).reshape(-1,1)
dT_dt_DJF_10 = np.mean(dT_dt_DJF_10).reshape(-1,1)
dT_dt_DJF_11 = np.mean(dT_dt_DJF_11).reshape(-1,1)
dT_dt_DJF_12 = np.mean(dT_dt_DJF_12).reshape(-1,1)
dT_dt_DJF_13 = np.mean(dT_dt_DJF_13).reshape(-1,1)
dT_dt_DJF_14 = np.mean(dT_dt_DJF_14).reshape(-1,1)
dT_dt_DJF_15 = np.mean(dT_dt_DJF_15).reshape(-1,1)
dT_dt_DJF_16 = np.mean(dT_dt_DJF_16).reshape(-1,1)
dT_dt_DJF_17 = np.mean(dT_dt_DJF_17).reshape(-1,1)
dT_dt_DJF_18 = np.mean(dT_dt_DJF_18).reshape(-1,1)
dT_dt_DJF_19 = np.mean(dT_dt_DJF_19).reshape(-1,1)

In [127]:
R_recur_DJF_93 = np.mean(R_recur_DJF_93, axis = (1,2)).reshape(-1,1)
R_recur_DJF_94 = np.mean(R_recur_DJF_94, axis = (1,2)).reshape(-1,1)
R_recur_DJF_95 = np.mean(R_recur_DJF_95, axis = (1,2)).reshape(-1,1)
R_recur_DJF_96 = np.mean(R_recur_DJF_96, axis = (1,2)).reshape(-1,1)
R_recur_DJF_97 = np.mean(R_recur_DJF_97, axis = (1,2)).reshape(-1,1)
R_recur_DJF_98 = np.mean(R_recur_DJF_98, axis = (1,2)).reshape(-1,1)
R_recur_DJF_99 = np.mean(R_recur_DJF_99, axis = (1,2)).reshape(-1,1)
R_recur_DJF_00 = np.mean(R_recur_DJF_00, axis = (1,2)).reshape(-1,1)
R_recur_DJF_01 = np.mean(R_recur_DJF_01, axis = (1,2)).reshape(-1,1)
R_recur_DJF_02 = np.mean(R_recur_DJF_02, axis = (1,2)).reshape(-1,1)
R_recur_DJF_03 = np.mean(R_recur_DJF_03, axis = (1,2)).reshape(-1,1)
R_recur_DJF_04 = np.mean(R_recur_DJF_04, axis = (1,2)).reshape(-1,1)
R_recur_DJF_05 = np.mean(R_recur_DJF_05, axis = (1,2)).reshape(-1,1)
R_recur_DJF_06 = np.mean(R_recur_DJF_06, axis = (1,2)).reshape(-1,1)
R_recur_DJF_07 = np.mean(R_recur_DJF_07, axis = (1,2)).reshape(-1,1)
R_recur_DJF_08 = np.mean(R_recur_DJF_08, axis = (1,2)).reshape(-1,1)
R_recur_DJF_09 = np.mean(R_recur_DJF_09, axis = (1,2)).reshape(-1,1)
R_recur_DJF_10 = np.mean(R_recur_DJF_10, axis = (1,2)).reshape(-1,1)
R_recur_DJF_11 = np.mean(R_recur_DJF_11, axis = (1,2)).reshape(-1,1)
R_recur_DJF_12 = np.mean(R_recur_DJF_12, axis = (1,2)).reshape(-1,1)
R_recur_DJF_13 = np.mean(R_recur_DJF_13, axis = (1,2)).reshape(-1,1)
R_recur_DJF_14 = np.mean(R_recur_DJF_14, axis = (1,2)).reshape(-1,1)
R_recur_DJF_15 = np.mean(R_recur_DJF_15, axis = (1,2)).reshape(-1,1)
R_recur_DJF_16 = np.mean(R_recur_DJF_16, axis = (1,2)).reshape(-1,1)
R_recur_DJF_17 = np.mean(R_recur_DJF_17, axis = (1,2)).reshape(-1,1)
R_recur_DJF_18 = np.mean(R_recur_DJF_18, axis = (1,2)).reshape(-1,1)
R_recur_DJF_19 = np.mean(R_recur_DJF_19, axis = (1,2)).reshape(-1,1)

In [128]:
R_recur_DJF_93 = np.mean(R_recur_DJF_93).reshape(-1,1)
R_recur_DJF_94 = np.mean(R_recur_DJF_94).reshape(-1,1)
R_recur_DJF_95 = np.mean(R_recur_DJF_95).reshape(-1,1)
R_recur_DJF_96 = np.mean(R_recur_DJF_96).reshape(-1,1)
R_recur_DJF_97 = np.mean(R_recur_DJF_97).reshape(-1,1)
R_recur_DJF_98 = np.mean(R_recur_DJF_98).reshape(-1,1)
R_recur_DJF_99 = np.mean(R_recur_DJF_99).reshape(-1,1)
R_recur_DJF_00 = np.mean(R_recur_DJF_00).reshape(-1,1)
R_recur_DJF_01 = np.mean(R_recur_DJF_01).reshape(-1,1)
R_recur_DJF_02 = np.mean(R_recur_DJF_02).reshape(-1,1)
R_recur_DJF_03 = np.mean(R_recur_DJF_03).reshape(-1,1)
R_recur_DJF_04 = np.mean(R_recur_DJF_04).reshape(-1,1)
R_recur_DJF_05 = np.mean(R_recur_DJF_05).reshape(-1,1)
R_recur_DJF_06 = np.mean(R_recur_DJF_06).reshape(-1,1)
R_recur_DJF_07 = np.mean(R_recur_DJF_07).reshape(-1,1)
R_recur_DJF_08 = np.mean(R_recur_DJF_08).reshape(-1,1)
R_recur_DJF_09 = np.mean(R_recur_DJF_09).reshape(-1,1)
R_recur_DJF_10 = np.mean(R_recur_DJF_10).reshape(-1,1)
R_recur_DJF_11 = np.mean(R_recur_DJF_11).reshape(-1,1)
R_recur_DJF_12 = np.mean(R_recur_DJF_12).reshape(-1,1)
R_recur_DJF_13 = np.mean(R_recur_DJF_13).reshape(-1,1)
R_recur_DJF_14 = np.mean(R_recur_DJF_14).reshape(-1,1)
R_recur_DJF_15 = np.mean(R_recur_DJF_15).reshape(-1,1)
R_recur_DJF_16 = np.mean(R_recur_DJF_16).reshape(-1,1)
R_recur_DJF_17 = np.mean(R_recur_DJF_17).reshape(-1,1)
R_recur_DJF_18 = np.mean(R_recur_DJF_18).reshape(-1,1)
R_recur_DJF_19 = np.mean(R_recur_DJF_19).reshape(-1,1)

In [129]:
Q_net_DJF_93.shape

(1, 1)

In [130]:
Q_net_DJF_93_19 = np.concatenate((Q_net_DJF_93, Q_net_DJF_94, Q_net_DJF_95, Q_net_DJF_96,
                                 Q_net_DJF_97, Q_net_DJF_98, Q_net_DJF_99, Q_net_DJF_00, Q_net_DJF_01,
                                 Q_net_DJF_02, Q_net_DJF_03, Q_net_DJF_04, Q_net_DJF_05, Q_net_DJF_06,
                                 Q_net_DJF_07, Q_net_DJF_08, Q_net_DJF_09, Q_net_DJF_10, Q_net_DJF_11,
                                 Q_net_DJF_12, Q_net_DJF_13, Q_net_DJF_14, Q_net_DJF_15, Q_net_DJF_16,
                                 Q_net_DJF_17, Q_net_DJF_18, Q_net_DJF_19), axis = 0)
Q_net_DJF_93_19.shape

(27, 1)

In [131]:
ZAdv_DJF_93_19 = np.concatenate((ZAdv_DJF_93, ZAdv_DJF_94, ZAdv_DJF_95, ZAdv_DJF_96,
                                 ZAdv_DJF_97, ZAdv_DJF_98, ZAdv_DJF_99, ZAdv_DJF_00, ZAdv_DJF_01,
                                 ZAdv_DJF_02, ZAdv_DJF_03, ZAdv_DJF_04, ZAdv_DJF_05, ZAdv_DJF_06,
                                 ZAdv_DJF_07, ZAdv_DJF_08, ZAdv_DJF_09, ZAdv_DJF_10, ZAdv_DJF_11,
                                 ZAdv_DJF_12, ZAdv_DJF_13, ZAdv_DJF_14, ZAdv_DJF_15, ZAdv_DJF_16,
                                 ZAdv_DJF_17, ZAdv_DJF_18, ZAdv_DJF_19), axis = 0)
ZAdv_DJF_93_19.shape

(27, 1)

In [132]:
MAdv_DJF_93_19 = np.concatenate((MAdv_DJF_93, MAdv_DJF_94, MAdv_DJF_95, MAdv_DJF_96,
                                 MAdv_DJF_97, MAdv_DJF_98, MAdv_DJF_99, MAdv_DJF_00, MAdv_DJF_01,
                                 MAdv_DJF_02, MAdv_DJF_03, MAdv_DJF_04, MAdv_DJF_05, MAdv_DJF_06,
                                 MAdv_DJF_07, MAdv_DJF_08, MAdv_DJF_09, MAdv_DJF_10, MAdv_DJF_11,
                                 MAdv_DJF_12, MAdv_DJF_13, MAdv_DJF_14, MAdv_DJF_15, MAdv_DJF_16,
                                 MAdv_DJF_17, MAdv_DJF_18, MAdv_DJF_19), axis = 0)
MAdv_DJF_93_19.shape

(27, 1)

In [133]:
VAdv_DJF_93_19 = np.concatenate((VAdv_DJF_93, VAdv_DJF_94, VAdv_DJF_95, VAdv_DJF_96,
                                 VAdv_DJF_97, VAdv_DJF_98, VAdv_DJF_99, VAdv_DJF_00, VAdv_DJF_01,
                                 VAdv_DJF_02, VAdv_DJF_03, VAdv_DJF_04, VAdv_DJF_05, VAdv_DJF_06,
                                 VAdv_DJF_07, VAdv_DJF_08, VAdv_DJF_09, VAdv_DJF_10, VAdv_DJF_11,
                                 VAdv_DJF_12, VAdv_DJF_13, VAdv_DJF_14, VAdv_DJF_15, VAdv_DJF_16,
                                 VAdv_DJF_17, VAdv_DJF_18, VAdv_DJF_19), axis = 0)
VAdv_DJF_93_19.shape

(27, 1)

In [134]:
R_origin_DJF_93_19  = np.concatenate((R_origin_DJF_93 , R_origin_DJF_94, R_origin_DJF_95, R_origin_DJF_96,
                                 R_origin_DJF_97, R_origin_DJF_98, R_origin_DJF_99, R_origin_DJF_00, R_origin_DJF_01,
                                 R_origin_DJF_02, R_origin_DJF_03, R_origin_DJF_04, R_origin_DJF_05, R_origin_DJF_06,
                                 R_origin_DJF_07, R_origin_DJF_08, R_origin_DJF_09, R_origin_DJF_10, R_origin_DJF_11,
                                 R_origin_DJF_12, R_origin_DJF_13, R_origin_DJF_14, R_origin_DJF_15, R_origin_DJF_16,
                                 R_origin_DJF_17, R_origin_DJF_18, R_origin_DJF_19), axis = 0)
R_origin_DJF_93_19.shape

(27, 1)

In [135]:
dT_dt_DJF_93_19  = np.concatenate((dT_dt_DJF_93 , dT_dt_DJF_94, dT_dt_DJF_95, dT_dt_DJF_96,
                                 dT_dt_DJF_97, dT_dt_DJF_98, dT_dt_DJF_99, dT_dt_DJF_00, dT_dt_DJF_01,
                                 dT_dt_DJF_02, dT_dt_DJF_03, dT_dt_DJF_04, dT_dt_DJF_05, dT_dt_DJF_06,
                                 dT_dt_DJF_07, dT_dt_DJF_08, dT_dt_DJF_09, dT_dt_DJF_10, dT_dt_DJF_11,
                                 dT_dt_DJF_12, dT_dt_DJF_13, dT_dt_DJF_14, dT_dt_DJF_15, dT_dt_DJF_16,
                                 dT_dt_DJF_17, dT_dt_DJF_18, dT_dt_DJF_19), axis = 0)
dT_dt_DJF_93_19.shape

(27, 1)

In [136]:
R_recur_DJF_93_19  = np.concatenate((R_recur_DJF_93 , R_recur_DJF_94, R_recur_DJF_95, R_recur_DJF_96,
                                 R_recur_DJF_97, R_recur_DJF_98, R_recur_DJF_99, R_recur_DJF_00, R_recur_DJF_01,
                                 R_recur_DJF_02, R_recur_DJF_03, R_recur_DJF_04, R_recur_DJF_05, R_recur_DJF_06,
                                 R_recur_DJF_07, R_recur_DJF_08, R_recur_DJF_09, R_recur_DJF_10, R_recur_DJF_11,
                                 R_recur_DJF_12, R_recur_DJF_13, R_recur_DJF_14, R_recur_DJF_15, R_recur_DJF_16,
                                 R_recur_DJF_17, R_recur_DJF_18, R_recur_DJF_19), axis = 0)
R_recur_DJF_93_19.shape

(27, 1)

In [138]:
np.savez(r'D:\heat_wave\WEIO\expand_WEIO\last\Figure1_MLD_eq\MLD_equa_DJF.npz',
        Q_net_DJF_93_19 = Q_net_DJF_93_19, ZAdv_DJF_93_19 = ZAdv_DJF_93_19,
        MAdv_DJF_93_19 = MAdv_DJF_93_19, VAdv_DJF_93_19 = VAdv_DJF_93_19,
        R_origin_DJF_93_19 = R_origin_DJF_93_19, dT_dt_DJF_93_19 = dT_dt_DJF_93_19,
        R_recur_DJF_93_19 = R_recur_DJF_93_19)